https://towardsdatascience.com/automate-your-mundane-excel-reporting-with-python-f3a29e6e3a0a

In [1]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.chart import Reference, BarChart

# Section 1: Gathering files
path = '../data/Excel Files/'
files = os.listdir(path)

# Section 2: Combining Files
combined = pd.DataFrame()
for file in files:
    df = pd.read_excel(path+file)
    combined = pd.concat([combined, df], ignore_index=True)

combined.to_excel('/home/lucas/Área de Trabalho/Combined.xlsx')

In [7]:
# Section 3: Summarizing Data
summary = pd.pivot_table(
    data=combined,
    index='Salesperson',
    values='Amount',
    aggfunc='sum'
)

summaryxls = '/home/lucas/Área de Trabalho/Summary.xlsx'

summary.to_excel(summaryxls)

In [8]:

# Section 4: Adding Descriptive Titles
# Loading Workbook and Worksheet Objects
wb = load_workbook(summaryxls)
ws = wb['Sheet1']

# Inserting Descriptive Titles
ws.insert_rows(0, 3)
ws['A1'].value = 'Sales by Salesperson'
ws['A2'].value = 'datagy.io'

# Saving the Workbook
wb.save(summaryxls)

In [9]:
# Section 5: Creating Reference Objects
data = Reference(ws, min_col=2, max_col=2, min_row=5, max_row=ws.max_row)
categories = Reference(ws, min_col=1, max_col=1, min_row=5, max_row=ws.max_row)

# Section 6: Adding a Bar Chart to Our Workbook
chart = BarChart()
chart.add_data(data)
chart.set_categories(categories)
ws.add_chart(chart, anchor='F4')

wb.save(summaryxls)

In [10]:
# Section 7: Styling Cells
ws['A1'].style = 'Title'
ws['A2'].style = 'Headline 2'

for cell in range(5, ws.max_row+1):
    ws[f'B{cell}'].style = 'Currency'
    
wb.save(summaryxls)

* How can you add names to worksheets?
* How can you email your resulting file automatically?
* How can you style values as tables?

In [ ]:
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo

wb = Workbook()
ws = wb.active

data = [
    ['Apples', 10000, 5000, 8000, 6000],
    ['Pears',   2000, 3000, 4000, 5000],
    ['Bananas', 6000, 6000, 6500, 6000],
    ['Oranges',  500,  300,  200,  700],
]

# add column headings. NB. these must be strings
ws.append(["Fruit", "2011", "2012", "2013", "2014"])
for row in data:
    ws.append(row)

tab = Table(displayName="Table1", ref="A1:E5")

# Add a default style with striped rows and banded columns
style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=True)
tab.tableStyleInfo = style

'''
Table must be added using ws.add_table() method to avoid duplicate names.
Using this method ensures table name is unque through out defined names and all other table name. 
'''
ws.add_table(tab)
wb.save("table.xlsx")